##3.3. 线性回归的简介实现
&emsp;&emsp;随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如果使用tensorflow提供的keras模块更方便地实现线性回归的训练。

In [0]:
%matplotlib inline
from IPython import display
from matplotlib import pyplot as pyplot
import tensorflow as tf
import random
import numpy as np
import tensorflow.keras as keras

tf.enable_eager_execution() #启用动态图计算

###3.3.1.生成数据集
&emsp;&emsp;我们生成与上一节中相同的数据集。其中features是训练数据特征，labels是标签。

In [0]:
num_inputs=2
num_examples=1000
true_w=[2,-3.4]
true_b=4.2
features=tf.random.normal(shape=(num_examples,num_inputs),stddev=1)
labels=true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels+=tf.random.normal(shape=labels.shape,stddev=0.01)

In [9]:
features.numpy().shape,labels.numpy().shape

((1000, 2), (1000,))

###3.3.2.读取数据
&emsp;&emsp;我们使用tensorflow中的data模块中的Dataset类来读取数据。Dataset.from_tensor_slices()方法传入张量并返回Dataset实例。设置batch_size为10，在每一次迭代中，我们将随机读取包含10个数据样本的小批量。

In [0]:
batch_size=10
buffer_size=1000
dataset=tf.data.Dataset.from_tensor_slices((features,labels))
dataset=dataset.shuffle(1000).batch(batch_size)

In [11]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [12]:
for x,y in dataset:
  print(x.shape,y.shape)
  break

(10, 2) (10,)


### 3.3.3定义模型
&emsp;&emsp;
在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤变得更繁琐。其实，keras模块提供了大量预定义的层，这使我们只需要关注用哪些层来构造模型。下面将介绍如何使用keras更简洁地定义线性回归。\
&emsp;&emsp;首先，从keras中导入Sequential类，可以将它看作是一个串联各个层的容器。在构造模型时，我们在该容器中依次添加层。当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。

In [0]:
from tensorflow.keras import Sequential

In [0]:
net=Sequential()

&emsp;&emsp;回顾图3.1中线性回归在神经网络图中的表示。作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，在线性回归的输出层又叫全连接层。在keras中，全连接层是一个Dense实例。我们定义该层输出个数为1.

In [0]:
net.add(keras.layers.Dense(1))

&emsp;&emsp;值得一提的是，在keras中我们无须指定每一层输入的形状，例如线性回归的输入个数。当模型得到数据时，例如后面执行net(X)时，模型将自动推断出每一层的输入个数。

###3.3.4 定义损失函数
&emsp;&emsp;在keras中，losses模块定义了各种损失。我们使用平方误差损失作为模型的损失函数。

In [0]:
loss=keras.losses.mean_squared_error

###3.3.5 定义优化算法
&emsp;&emsp;同样，我们也无须实现小批量随机梯度下降。在导入keras后，使用optimizer模块创建一个SGD实例，并指定学习率为0.03。该优化算法将用来迭代net实例所有通过add函数嵌套的层所包含的全部参数。

In [0]:
optimizer=keras.optimizers.SGD(learning_rate=0.03) #定义优化器

###3.3.6. 训练模型
&emsp;&emsp;在使用keras训练模型时，需要先调用compile()函数来编译模型，在该函数中指定optimizer优化器、损失函数已经评价指标。然后调用fit_generator()函数指定训练数据和epoch进行训练。

In [18]:
num_epochs=3
net.compile(optimizer=optimizer,loss=loss,metrics=[keras.metrics.mae])
net.fit_generator(dataset,epochs=3)

Epoch 1/3
100/100 [==============================] - 1s 9ms/step - loss: 3.2955 - mean_absolute_error: 2.0419
Epoch 2/3
100/100 [==============================] - 1s 6ms/step - loss: 1.1064e-04 - mean_absolute_error: 0.0091
Epoch 3/3
100/100 [==============================] - 1s 6ms/step - loss: 1.0312e-04 - mean_absolute_error: 0.0081


&emsp;&emsp;让我们打印出模型结构看看，可以看到模型仅包含一个Dense层，并且参数数量为3.

In [19]:
net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


&emsp;&emsp;下面我们比较学习到的模型参数和真实的模型参数。我们从net.get_weights()函数获取权重（weight）和偏差(bias)。学到的参数和真实的参数很接近

In [20]:
train_w,train_b=net.get_weights()
true_w,train_w

([2, -3.4], array([[ 2.0006943],
        [-3.399984 ]], dtype=float32))

In [21]:
true_b,train_b

(4.2, array([4.1989474], dtype=float32))

###3.3.7. 小结
* 在keras中可以更简洁地实现模型。
* tf.data模块提供了数据处理的工具，keras中定义了大量的神经网络层，keras.losses模块定义了各种损失函数。keras.optimizers模块中定义了各种优化算法。